<a href="https://colab.research.google.com/github/dapopov-st/ExperimentsWithLanguageModels/blob/main/RAG_over_ArXiv_PDFs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

I'm using [Chris Alexiuk's](https://www.linkedin.com/in/csalexiuk/) [notebook](https://colab.research.google.com/drive/172uMprWwUfEecXQWBrsgDAlkpT_EK39z?usp=sharing)
as a starting point and plan to experiment with some of the ideas from [A Practical Approach to Retrieval Augmented Generation Systems](https://angelinamagr.gumroad.com/l/practical-approach-to-RAG-systems) by Allahyari and Yang.

## Steps
- Make a directory of ArXiv papers on Google Drive
- See if can store the vector store in Drive or just in ram
- Bridge this nb with the book
- If can't access A100 consistenly and run out of GPU memory, make a Mistral 7B/Zephyr 7B version (Zephyr can do ReAct!)
- Rag with metadata (ex, subdirectory name etc.)
- Use Self-Rag 7b? selfrag/selfrag_llama2_7b or 13b.  Think this will help overcome some of the challenges with retrieving from pdfs (getting titles from References section, for example, rather than useful content). UPDATE: fixed this by only including article content up to the References.
- Experiment with different vector stores.  FAISS seems to work for the usecase, but does not allow searching/filtering using metadata.  Consider using Pinecone or Elasticsearch, perhaps.

## Get the data and build a Retriever

- Original NB worked in under 10GB on V100

In [1]:
!pip install -U -q "langchain" "transformers==4.31.0" "datasets==2.13.0" "peft==0.4.0" "accelerate==0.21.0" "bitsandbytes==0.40.2" "trl==0.4.7" "safetensors>=0.3.1"
!pip install -U -q cohere llama-index
!pip install PyPDF2
!pip install pypdf
!pip install -q qdrant-client
!pip install -q -U faiss-cpu tiktoken sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 45.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
from google.colab import drive
drive.mount('/content/drive/')

output_dir = '/content/drive/MyDrive/PdfRag/rag_output_dir'
logging_dir = '/content/drive/MyDrive/PdfRag/rag_logging_dir'
index_dir = '/content/drive/MyDrive/PdfRag/rag_index_dir'

#!ls /content/drive/MyDrive/PdfRag/clusterofstars
%cd /content/drive/MyDrive/PdfRag
#My\ Drive/PdfRag && ls clusterofstars
!ls .

barbie.csv  cache  clusterofstars  oppenheimer.csv  rag_index_dir


- The documents consist of a few dozen ArXiv papers about modern LLMs

In [3]:
from pathlib import Path
PDFS_PATH = Path('/content/drive/MyDrive/PdfRag/clusterofstars')
PDFS = list(PDFS_PATH.glob('*.pdf'))
PDFS[0], len(PDFS)

(PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/InContextRALM.pdf'),
 28)

In [4]:
# For now, flatten out the file structure for simplicity.  In the future, see if can have Rag use this metadata.
#!mv /content/drive/MyDrive/PdfRag/clusterofstars/ChainOfThought/* /content/drive/MyDrive/PdfRag/clusterofstars

In [4]:
# fastai function to clean GPU memory

import sys,gc,traceback
import torch
def clean_ipython_hist():
    # Code in this function mainly copied from IPython source
    if not 'get_ipython' in globals(): return
    ip = get_ipython()
    user_ns = ip.user_ns
    ip.displayhook.flush()
    pc = ip.displayhook.prompt_count + 1
    for n in range(1, pc): user_ns.pop('_i'+repr(n),None)
    user_ns.update(dict(_i='',_ii='',_iii=''))
    hm = ip.history_manager
    hm.input_hist_parsed[:] = [''] * pc
    hm.input_hist_raw[:] = [''] * pc
    hm._i = hm._ii = hm._iii = hm._i00 =  ''



def clean_tb():
    # h/t Piotr Czapla
    if hasattr(sys, 'last_traceback'):
        traceback.clear_frames(sys.last_traceback)
        delattr(sys, 'last_traceback')
    if hasattr(sys, 'last_type'): delattr(sys, 'last_type')
    if hasattr(sys, 'last_value'): delattr(sys, 'last_value')

def clean_mem():
    clean_tb()
    clean_ipython_hist()
    gc.collect()
    torch.cuda.empty_cache()



### Task 1: Prepare the data and  build a PDF Data Loader

In [5]:
!pip install --q pdfminer.six

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 23.8 MB/s eta 0:00:00


In [28]:
from langchain.document_loaders import PDFMinerLoader

In [29]:
loader = PDFMinerLoader(os.path.expanduser(PDFS[0]))

In [30]:
pdf_content = loader.load()
#pdf_content

In [31]:
from PyPDF2 import PdfReader
reader = PdfReader(os.path.expanduser(PDFS[0]))
pages = reader.pages
documents = []
for page in pages:
  documents.append(page.extract_text())
#print(documents[-1])

## First drop everything from References onwards. References were 'confusing' RAG into retrieving primarily titles of papers mentioned there, which is likely not very useful

In [140]:
import PyPDF2

def load_pdf_to_string(pdf_path):
    # Open the PDF file in binary mode
    with open(pdf_path, 'rb') as file:
        # Create a PDF file reader object
        pdf_reader = PyPDF2.PdfReader(file)

        # Initialize an empty string to hold the text
        text = ''

        # Loop through each page and extract the text
        for page_num in range(len(pdf_reader.pages)):
            page = pdf_reader.pages[page_num]
            page_text = page.extract_text()
            references_index= page_text.upper().find('\nREFERENCES\n')
            if references_index != -1:
              page_text = page_text[:references_index]
              text += page_text
              return text
            text += page_text
    return text

# Use the function to load a PDF into a string
text = load_pdf_to_string(os.path.expanduser(PDFS[1]))

In [118]:
text.find('References\n')

-1

In [90]:
PDFS[0]

PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/InContextRALM.pdf')

In [141]:
all_docs = [load_pdf_to_string(os.path.expanduser(pdf_path)) for pdf_path in PDFS]

In [120]:
len(all_docs)

28

In [121]:
all_docs[-1].find('References\n')

-1

In [123]:
all_docs[0]

'In-Context Retrieval-Augmented Language Models\nOri Ram∗Yoav Levine∗Itay Dalmedigos Dor Muhlgay\nAmnon Shashua Kevin Leyton-Brown Yoav Shoham\nAI21 Labs\n{orir,yoavl,itayd,dorm,amnons,kevinlb,yoavs}@ai21.com\nAbstract\nRetrieval-Augmented Language Modeling\n(RALM) methods, which condition a lan-\nguage model (LM) on relevant documents\nfrom a grounding corpus during generation,\nwere shown to signi\ue000cantly improve lan-\nguage modeling performance. In addition,\nthey can mitigate the problem of factually\ninaccurate text generation and provide natu-\nral source attribution mechanism. Existing\nRALM approaches focus on modifying the\nLM architecture in order to facilitate the in-\ncorporation of external information, signi\ue000-\ncantly complicating deployment. This paper\nconsiders a simple alternative, which we dub\nIn-Context RALM: leaving the LM architec-\nture unchanged and prepending grounding\ndocuments to the input,without any further\ntraining of the LM. We show that In-Co

In [65]:
# from langchain.document_loaders import PyPDFLoader

# loader = PyPDFLoader(os.path.expanduser(PDFS[0]))
# #pages = loader.load_and_split()
# pages = loader.load()
# all_pages = [PyPDFLoader(os.path.expanduser(PDFS[i])).load() for i in range(len(PDFS))]

- Instead of load_and_split, going for a more custom split to have more control over context length sicnce working with limited compute

In [66]:
#pages[0], len(pages)

-Either PyPDFLoader or PyPDF2 approach could work

In [67]:
# from langchain.document_loaders.onedrive_file import CHUNK_SIZE
# from langchain.document_loaders import TextLoader
# from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter
# CHUNK_SIZE = 1000
# CHUNK_OVERLAP = 30

# text_splitter = CharacterTextSplitter(
#     chunk_size=CHUNK_SIZE,
#     chunk_overlap = CHUNK_OVERLAP,
#     length_function=len,
# )
# docs  = text_splitter.split_documents(pages)
# docs

FROM BARBIEHEIMER

Now that we have collected our review information into a loader - we can go ahead and chunk the reviews into more manageable pieces.

We'll be leveraging the `RecursiveCharacterTextSplitter` for this task today.

While splitting our text seems like a simple enough task - getting this correct/incorrect can have massive downstream impacts on your application's performance.

You can read the docs here:
- [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/text_splitters/recursive_text_splitter)

> ### HINT:
>It's always worth it to check out the LangChain source code if you're ever in a bind - for instance, if you want to know how to transform a set of documents, check it out [here](https://github.com/langchain-ai/langchain/blob/5e9687a196410e9f41ebcd11eb3f2ca13925545b/libs/langchain/langchain/text_splitter.py#L268C18-L268C18)

In [146]:
from langchain.document_loaders.onedrive_file import CHUNK_SIZE
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter, TokenTextSplitter, RecursiveCharacterTextSplitter
from langchain.text_splitter import Document

# CHUNK_SIZE = 1000
# CHUNK_OVERLAP = 30
CHUNK_SIZE = 1000
CHUNK_OVERLAP = 30

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap = CHUNK_OVERLAP,
    length_function=len,
)
#text_splitter.split_text(all_pages[0])
# docs = [Document(page_content=pages) for pages in all_pages]
docs  = [text_splitter.split_text(doc) for doc in all_docs]
# # docs

In [125]:
len(docs)

28

In [127]:
docs[0]

['In-Context Retrieval-Augmented Language Models\nOri Ram∗Yoav Levine∗Itay Dalmedigos Dor Muhlgay\nAmnon Shashua Kevin Leyton-Brown Yoav Shoham\nAI21 Labs\n{orir,yoavl,itayd,dorm,amnons,kevinlb,yoavs}@ai21.com\nAbstract\nRetrieval-Augmented Language Modeling\n(RALM) methods, which condition a lan-\nguage model (LM) on relevant documents\nfrom a grounding corpus during generation,\nwere shown to signi\ue000cantly improve lan-\nguage modeling performance. In addition,\nthey can mitigate the problem of factually\ninaccurate text generation and provide natu-\nral source attribution mechanism. Existing\nRALM approaches focus on modifying the\nLM architecture in order to facilitate the in-\ncorporation of external information, signi\ue000-\ncantly complicating deployment. This paper\nconsiders a simple alternative, which we dub\nIn-Context RALM: leaving the LM architec-\nture unchanged and prepending grounding\ndocuments to the input,without any further\ntraining of the LM. We show that In-C

In [86]:
PDFS[-1]

PosixPath('/content/drive/MyDrive/PdfRag/clusterofstars/Self-Rag.pdf')

In [16]:
len(docs)

28

In [17]:
len(docs[0])

62

### Task 2: Create an "Index"

The term "index" is used largely to mean: Structured documents parsed into a useful format for querying, retrieving, and use in the LLM application stack.

- Not yet sure if should use Qdrant or FAISS


#### Selecting the VectorStore

There are a number of different VectorStores, and a number of different strengths and weaknesses to each.

In this notebook, we will be keeping it very simple by leveraging [Facebook AI Similarity Search](https://ai.meta.com/tools/faiss/#:~:text=FAISS%20(Facebook%20AI%20Similarity%20Search,more%20scalable%20similarity%20search%20functions.), or `FAISS`.

In [6]:
from langchain.vectorstores import Qdrant, FAISS

FROM BARBIEHEIMER

We're going to be setting up our VectorStore with the OpenAI embeddings model. While this embeddings model does not need to be consistent with the LLM selection, it does need to be consistent between embedding our index and embedding our queries over that index.

While we don't have to worry too much about that in this example - it's something to keep in mind for more complex applications.

We're going to leverage a [`CacheBackedEmbeddings`](https://python.langchain.com/docs/modules/data_connection/caching_embeddings )flow to prevent us from re-embedding similar queries over and over again.

Not only will this save time, it will also save us precious embedding tokens, which will reduce the overall cost for our application.

>#### Note:
>The overall cost savings needs to be compared against the additional cost of storing the cached embeddings for a true cost/benefit analysis. If your users are submitting the same queries often, though, this pattern can be a massive reduction in cost.

In [129]:
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore

store = LocalFileStore("./cache/")

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)

embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)


In [147]:
#vector_store = FAISS.from_documents(docs, embedder)
#vector_store = FAISS.from_documents((docs[i][j] for i in range(len(docs)) for j in range(len(docs[i]))), embedder)
#vector_store = FAISS.from_documents(docs, embedder)
#vector_store = FAISS.from_documents(docs[0], embedder)
from langchain.schema.document import Document

docs = [Document(page_content=doc[i]) for doc in docs for i in range(len(doc))]
for index, pdf in enumerate(docs):
   content = docs[index]
   if index == 0:
       vector_store = FAISS.from_documents([content], embedder)
   else:
      vector_store_i = FAISS.from_documents([content], embedder)
      vector_store.merge_from(vector_store_i)


vector_store.save_local(index_dir)

- To reload the embeddings made above on the next Colab nb use, run the code below.

In [10]:
from langchain.embeddings import CacheBackedEmbeddings, HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
store = LocalFileStore("./cache/")

embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'

core_embeddings_model = HuggingFaceEmbeddings(
    model_name=embed_model_id
)
embedder = CacheBackedEmbeddings.from_bytes_store(
    core_embeddings_model, store, namespace=embed_model_id
)

vector_store = FAISS.load_local(index_dir, embedder)

(…)f3d3c277d6e90027e55de9125/.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

(…)7d6e90027e55de9125/1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(…)e2f80f3d3c277d6e90027e55de9125/README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

(…)f80f3d3c277d6e90027e55de9125/config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

(…)de9125/config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

(…)f3d3c277d6e90027e55de9125/tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

(…)7d6e90027e55de9125/tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

(…)3d3c277d6e90027e55de9125/train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

(…)e2f80f3d3c277d6e90027e55de9125/vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

(…)80f3d3c277d6e90027e55de9125/modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

Check that the VectorStore is working by embedding a query and retrieving passages from our reviews that are close to it.

In [11]:
query = "What is Retrieval-augmented generation?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)

1 I NTRODUCTION
State-of-the-art LLMs continue to struggle with factual errors (Mallen et al., 2023; Min et al., 2023)
despite their increased model and data scale (Ouyang et al., 2022). Retrieval-Augmented Generation
(RAG) methods (Figure 1 left; Lewis et al. 2020; Guu et al. 2020) augment the input of LLMs
with relevant retrieved passages, reducing factual errors in knowledge-intensive tasks (Ram et al.,
2023; Asai et al., 2023a). However, these methods may hinder the versatility of LLMs or introduce
unnecessary or off-topic passages that lead to low-quality generations (Shi et al., 2023) since they
retrieve passages indiscriminately regardless of whether the factual grounding is helpful. Moreover,
the output is not guaranteed to be consistent with retrieved relevant passages (Gao et al., 2023) since
the models are not explicitly trained to leverage and follow facts from provided passages. This
work introduces Self-Reflective Retrieval-augmented Generation ( SELF-RAG)to improve an
LL

In [12]:
query = "What is Self-Rag?"
embedding_vector = core_embeddings_model.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

for page in docs:
  print(page.page_content)

predictions are mostly aligned with their assessments. Appendix Table 6 shows several annotated
examples and explanations on assessments.
6 C ONCLUSION
This work introduces SELF-RAG, a new framework to enhance the quality and factuality of LLMs
through retrieval on demand and self-reflection. SELF-RAGtrains an LM to learn to retrieve, generate,
and critique text passages and its own generation by predicting the next tokens from its original
vocabulary as well as newly added special tokens, called reflection tokens. SELF-RAGfurther enables
the tailoring of LM behaviors at test time by leveraging reflection tokens. Our holistic evaluations on
six tasks using multiple metrics demonstrate that SELF-RAGsignificantly outperforms LLMs with
more parameters or with conventional retrieval-augmented generation approaches.
10Preprint.
ETHICAL CONCERNS
This work aims to improve the factuality of LLM outputs, the lack of which continues to cause nu-
3 S ELF-RAG: LEARNING TO RETRIEVE , GENERATE AND C

Let's see how much time the `CacheBackedEmbeddings` pattern saves us:

In [14]:
%%timeit -n 1 -r 1
query = "What is Self-Rag?"
embedding_vector = embedder.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

11.7 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [15]:
%%timeit
query = "What is Self-Rag?"
embedding_vector = embedder.embed_query(query)
docs = vector_store.similarity_search_by_vector(embedding_vector, k = 4)

7.37 ms ± 223 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Looked like retrieving from References section may be a problem since references came up frequently for searched terms without being highly informative.  
- REMOVING REFERENCES ONWARDS INCREASED THE RETRIEVAL QUALITY NOTICEABLY
- TODO: Could experiment with different retriever or a smart-enough model (Self-Rag?)

As we can see, even over a significant number of runs - the cached query is significantly faster than the first instance of the query!

With that, we're ready to move onto Task 3!

### Task 3: Building a Retrieval Chain

In this task, we'll be making a Retrieval Chain which will allow us to ask semantic questions over our data.

This part is rather abstracted away from us in LangChain and so it seems very powerful.

Be sure to check the documentation, the source code, and other provided resources to build a deeper understanding of what's happening "under the hood"!

#### A Basic RetrievalQA Chain

We're going to leverage `return_source_documents=True` to ensure we have proper sources for our reviews - should the end user want to verify the reviews themselves.

Hallucinations [are](https://arxiv.org/abs/2202.03629) [a](https://arxiv.org/abs/2305.15852) [massive](https://arxiv.org/abs/2303.16104) [problem](https://arxiv.org/abs/2305.18248) in LLM applications.

Though it has been tenuously shown that using Retrieval Augmentation [reduces hallucination in conversations](https://arxiv.org/pdf/2104.07567.pdf), one sure fire way to ensure your model is not hallucinating in a non-transparent way is to provide sources with your responses. This way the end-user can verify the output.

#### Our LLM

In this notebook, we're going to leverage Meta's LLaMA 2!

Specifically, we'll be using: `meta-llama/Llama-2-13b-chat-hf`

That's right, a 13B parameter model that we're going to run on *less than* 15GB of GPU RAM.

More information on this model can be found [here](https://huggingface.co/meta-llama/Llama-2-13b-chat-hf)

In [16]:
!pip install huggingface-hub -q

In [17]:
from huggingface_hub import notebook_login

notebook_login()

We will be leveraging Tim Dettmer's `bitsandbytes` as well as `accelerate` and `transformers` from Hugging Face to make our model as small as possible. The overall quality of the model is fairly well retained!

In [18]:
import torch
import transformers

model_id = "meta-llama/Llama-2-13b-chat-hf"

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

model_config = transformers.AutoConfig.from_pretrained(
    model_id
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto'
)

model.eval()

(…)a-2-13b-chat-hf/resolve/main/config.json:   0%|          | 0.00/587 [00:00<?, ?B/s]

(…)esolve/main/model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

(…)t-hf/resolve/main/generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )


In [19]:
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id
)

(…)at-hf/resolve/main/tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)-13b-chat-hf/resolve/main/tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

(…)-hf/resolve/main/special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

Now we need to pack it into a `pipeline` for compatability with `langchain`!

In [20]:
!pip install xformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 4.1 MB/s eta 0:00:00


In [21]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    return_full_text=True,
    temperature=0.0,
    max_new_tokens=256
)

    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.1.0+cu118)
    Python  3.10.13 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [22]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

Now we can set up our chain.

In [23]:
retriever = vector_store.as_retriever()

In [24]:
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler

handler = StdOutCallbackHandler()

qa_with_sources_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    callbacks=[handler],
    return_source_documents=True
)

Now that it's set-up, let's test it out!

In [25]:
qa_with_sources_chain({"query" : "How does Self-Rag compare to Rag?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'How does Self-Rag compare to Rag?',
 'result': " Self-Rag is a new framework that enhances the quality and factuality of LLMs through retrieval and self-reflection, without sacrificing LLM's original creativity and versatility. It is different from conventional RAG approaches that retrieve passages indiscriminately, without ensuring complete support from cited sources. Self-Rag trains an LM to learn to retrieve, generate, and critique text passages and its own generation by predicting the next tokens from its original vocabulary as well as newly added special tokens, called reflection tokens.\n\nPlease let me know if you need any further information or context to answer the question.",
 'source_documents': [Document(page_content='performance. In particular, we randomly sample 5k, 10k, 20k, and 50k instances from our original\n150k training instances, and fine-tune four SELF-RAG 7Bvariants on those subsets. Then, we compare\nthe model performance on PopQA, PubHealth, and ASQA

In [26]:
qa_with_sources_chain({"query" : "What is QLoRa?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'What is QLoRa?',
 'result': ' QLoRa is a method for finetuning large language models that uses a combination of 4-bit and 16-bit data types to reduce the required memory for the model. It has been shown to be effective in producing state-of-the-art chatbots that rival ChatGPT.',
 'source_documents': [Document(page_content='and compare QLoRA with 16-bit adapter-finetuning and with full-finetuning for models up to 3B. Our\nevaluations include GLUE [ 58] with RoBERTa-large [ 38], Super-NaturalInstructions (TKInstruct)\n[61] with T5 [ 49], and 5-shot MMLU [ 24] after finetuning LLaMA on Flan v2 [ 39] and Alpaca\n[55]. To additionally study the advantages of NF4 over other 4-bit data types, we use the setup of\nDettmers and Zettlemoyer [13] and measure post-quantization zero-shot accuracy and perplexity\nacross different models (OPT [ 72], LLaMA [ 57], BLOOM [ 52], Pythia [ 7]) for model sizes 125m -\n13B. We provide more details in the results section for each particular setup t

In [27]:
qa_with_sources_chain({"query" : "Did these papers explore themes of existentialism?"})



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Did these papers explore themes of existentialism?',
 'result': ' No, these papers do not explore themes of existentialism.',
 'source_documents': [Document(page_content='members and other supporters of the Stanford DAWN project–Facebook, Google, and VMware—as\nwell as the NSF under CAREER grant CNS-1651570. Any opinions, findings, and conclusions or\nrecommendations expressed in this material are those of the authors and do not necessarily reflect\nthe views of the National Science Foundation. Omar Khattab is supported by the Apple Scholars in\nAI/ML fellowship.\n\\usepackage[pdftex]{graphicx} ...\n\\includegraphics[width=0.8\\linewidth]{myfile.pdf}'),
  Document(page_content='members and other supporters of the Stanford DAWN project–Facebook, Google, and VMware—as\nwell as the NSF under CAREER grant CNS-1651570. Any opinions, findings, and conclusions or\nrecommendations expressed in this material are those of the authors and do not necessarily reflect\nthe views of the Na